In [ ]:
# import pandas as pd

# # Đọc file Excel
# input_file = r"C:\Khue\TDN\data\interim\sanluong_chuanhoa_ver3.xlsx"
# # Đọc file Excel với engine='openpyxl'
# df = pd.read_excel(input_file, engine='openpyxl')


# # Lưu thành file Parquet
# output_file = r"C:\Khue\TDN\data\interim\sanluong_chuanhoa_ver3.parquet"

# # Lưu file parquet
# df.to_parquet(output_file, engine='pyarrow')

# print(f"Đã chuyển đổi thành công từ {input_file} sang {output_file}")

Đã chuyển đổi thành công từ C:\Khue\TDN\data\interim\sanluong_chuanhoa_ver3.xlsx sang C:\Khue\TDN\data\interim\sanluong_chuanhoa_ver3.parquet


In [ ]:
import pandas as pd
import warnings
import os
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
import calendar

warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
# Đọc file DGMS_BCSVH
# dgms_path = r"C:\Khue\TDN\data\interim\DGMS_BCSVH_ver2.parquet"
# df_dgms = pd.read_parquet(dgms_path)

# Đọc file Sanluong
sanluong_path = r"C:\Khue\TDN\data\interim\sanluong_chuanhoa_ver3.parquet"
df_sanluong = pd.read_parquet(sanluong_path)

Đã đọc thành công 2 file:
- DGMS_BCSVH: C:\Khue\TDN\data\interim\DGMS_BCSVH_ver2.parquet
- Sanluong: C:\Khue\TDN\data\interim\sanluong_chuanhoa_ver3.parquet


In [ ]:
# df_dgms['SL'] = df_dgms.loc[:, '00:30':'00:00'].sum(axis=1) / 2
# df_dgms.loc[:, '00:30':'SL'] *= 1000
# df_dgms.loc[df_dgms['Nhà máy'] == 'Nậm Mít Luông', 'TD_THAMCHIEU'] = 'BAN_CHAT'
# # Tạo dataframe mới với các cột cần thiết và nhóm theo 'TD_THAMCHIEU' và 'Ngày'
# df_dgms_sanluong = df_dgms[['TD_THAMCHIEU', 'Ngày', 'SL']].groupby(['TD_THAMCHIEU', 'Ngày'], as_index=False)['SL'].sum()

# # Hiển thị kết quả
# df_dgms_sanluong

,TD_THAMCHIEU,Ngày,SL
0,BAC_HA,2024.04.18,448380.0
1,BAC_HA,2024.04.19,702155.0
2,BAC_HA,2024.04.20,939125.0
3,BAC_HA,2024.04.21,30750.0
4,BAC_HA,2024.04.22,816209.0
...,...,...,...
2853,TUYEN_QUANG,2025.02.25,2321440.0
2854,TUYEN_QUANG,2025.02.26,2473855.0
2855,TUYEN_QUANG,2025.02.27,2397680.0
2856,TUYEN_QUANG,2025.02.28,2400590.0


In [ ]:
# null_TD_THAMCHIEU_unique = df_dgms[df_dgms['TD_THAMCHIEU'].isnull()]['Nhà máy'].unique()
# print(f"Có {len(null_TD_THAMCHIEU_unique)} giá trị unique của cột 'Nhà máy' tương ứng với giá trị null trong cột 'TD_THAMCHIEU' của df_dgms: {null_TD_THAMCHIEU_unique}")

Có 1 giá trị unique của cột 'Nhà máy' tương ứng với giá trị null trong cột 'TD_THAMCHIEU' của df_dgms: ['Nậm Pảng 2']


## 2. Vẽ biểu đồ cho dữ liệu sản lượng của NPC cung cấp

In [17]:
# ==================== CELL 1: XỬ LÝ DỮ LIỆU ====================
# Tính sản lượng của mỗi nhà máy trong một ngày và đổi về triệu kWh
df_sanluong['SL'] = df_sanluong.loc[:, '00:30':'00:00'].sum(axis=1) / (2*1000000)

# Tìm kiếm các giá trị unique của cột 'NMTD' tương ứng với giá trị null trong cột 'TD_THAMCHIEU' của df_sanluong
null_TD_THAMCHIEU_unique = df_sanluong[df_sanluong['TD_THAMCHIEU'].isnull()]['NMTD'].unique()
print(f"Có {len(null_TD_THAMCHIEU_unique)} giá trị unique của cột 'Nhà máy' tương ứng với giá trị null trong cột 'TD_THAMCHIEU' của df_sanluong: {null_TD_THAMCHIEU_unique}")

# Tạo dataframe mới với các cột cần thiết và nhóm theo 'TD_THAMCHIEU' và 'Ngày'
df_sanluong_plot = df_sanluong[['TD_THAMCHIEU', 'Ngày', 'SL']].groupby(['TD_THAMCHIEU', 'Ngày'], as_index=False)['SL'].sum()

# ==================== CELL 2: XỬ LÝ THỜI GIAN VÀ TỐI ƯU HÓA ====================
if not df_sanluong.empty:
    print("🔄 Xử lý thời gian và tối ưu DataFrame...")
    try:
        df_sanluong_plot['Ngày'] = pd.to_datetime(df_sanluong_plot['Ngày'])
        df_sanluong_plot['YEAR'] = df_sanluong_plot['Ngày'].dt.year
        df_sanluong_plot['MONTH_NUM'] = df_sanluong_plot['Ngày'].dt.month
        df_sanluong_plot['DAY_NUM'] = df_sanluong_plot['Ngày'].dt.day
        
        index_cols = ['TD_THAMCHIEU', 'YEAR', 'MONTH_NUM', 'DAY_NUM']
        df_sanluong_plot.set_index(index_cols, inplace=True)
        df_sanluong_plot.sort_index(inplace=True)
        print("✅ Đã xử lý thời gian và tối ưu index DataFrame.")
        
    except Exception as e:
        print(f"❌ Lỗi khi xử lý dữ liệu: {e}")
        df_sanluong_plot = pd.DataFrame()

# ==================== CELL 3: TẠO GIAO DIỆN VÀ BIỂU ĐỒ ====================
if not df_sanluong_plot.empty:
    print("🔄 Tạo giao diện tương tác...")
    
    try:
        td_list = sorted(df_sanluong_plot.index.get_level_values('TD_THAMCHIEU').unique())
        year_list = sorted(df_sanluong_plot.index.get_level_values('YEAR').unique())
    except Exception as e:
        print(f"❌ Lỗi khi lấy danh sách từ index: {e}")
        td_list, year_list = [], []

    def make_dropdown(options, description, width="250px", margin="0px 20px 0px 0px"):
        safe_options = list(options) if options is not None else []
        return widgets.Dropdown(
            options=safe_options,
            description=description,
            layout=widgets.Layout(
                width=width, 
                margin=margin,
                max_width=width,
                min_width=width
            ),
            style={
                "description_width": "auto",
                "text_overflow": "ellipsis"
            },
            disabled=not bool(safe_options)
        )

    # Tạo các dropdown cho tab ngày
    td_dropdown_day = make_dropdown(td_list, "TD_THAMCHIEU:", "280px")
    year_dropdown_day = make_dropdown(year_list, "Năm:", "120px")
    month_dropdown_day = make_dropdown(list(range(1, 13)), "Tháng:", "120px")
    
    # Tạo các dropdown cho tab tháng
    td_dropdown_month = make_dropdown(td_list, "TD_THAMCHIEU:", "280px")
    year_dropdown_month = make_dropdown(year_list, "Năm:", "120px")
    
    # Nhóm widget
    day_widgets = [td_dropdown_day, year_dropdown_day, month_dropdown_day]
    month_widgets = [td_dropdown_month, year_dropdown_month]

    # Vùng output cho biểu đồ
    out_day = widgets.Output()
    out_month = widgets.Output()

    def plot_daily(td, year, month):
        with out_day:
            clear_output(wait=True)
            
            if not all([td, year, month]):
                print("⚠️ Vui lòng chọn đủ TD_THAMCHIEU, Năm và Tháng.")
                return

            try:
                idx_query = (td, year, month)
                title_ext = f"{month}/{year}"

                filtered_df = df_sanluong_plot.loc[idx_query].reset_index()
                
                if filtered_df.empty:
                    print("ℹ️ Không có dữ liệu cho lựa chọn này.")
                    return

                fig = px.line(
                    filtered_df,
                    x='Ngày',
                    y='SL',
                    render_mode='webgl'
                )

                fig.update_layout(
                    title=None,
                    annotations=[dict(
                        text=f"<b>Sản lượng theo ngày - {td} - {title_ext}</b>",
                        xref="paper", yref="paper",
                        x=0.45, y=1.1,
                        xanchor="center", yanchor="top",
                        showarrow=False,
                        font=dict(size=18, color="black")
                    )],
                    template="plotly_white",
                    height=600,
                    hovermode="x unified",
                    xaxis_title="Thời gian",
                    yaxis_title="Sản lượng (triệu kWh)",
                    xaxis=dict(
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        title_font=dict(size=14, color="black", weight='bold'),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8,
                        title=None
                    ),
                    yaxis=dict(
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        rangemode="tozero",
                        title_font=dict(size=14, color="black", weight="bold"),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8
                    ),
                    margin=dict(l=60, r=40, t=80, b=80),
                    paper_bgcolor="white",
                    plot_bgcolor="#f0f8ff"
                )
                fig.add_annotation(
                    text="<b>Thời gian</b>",
                    xref="paper",
                    yref="paper",
                    x=0.44,
                    y=-0.15,
                    showarrow=False,
                    font=dict(size=14, color="black"),
                )
                fig.update_traces(
                    line=dict(width=2, color="#3366ff"),
                    hovertemplate="Sản lượng: %{y} triệu kWh"
                )

                fig.show()

            except Exception as e:
                print(f"❌ Lỗi khi vẽ biểu đồ: {e}")

    def plot_monthly(td, year):
        with out_month:
            clear_output(wait=True)
            
            if not all([td, year]):
                print("⚠️ Vui lòng chọn TD_THAMCHIEU và Năm.")
                return

            try:
                # Lấy dữ liệu cho TD và năm được chọn
                filtered_df = df_sanluong_plot.loc[td, year]
                
                if filtered_df.empty:
                    print("ℹ️ Không có dữ liệu cho lựa chọn này.")
                    return

                # Tạo DataFrame với đầy đủ 12 tháng
                all_months = pd.DataFrame({
                    'MONTH_NUM': range(1, 13),
                    'Month': [f'T{i}' for i in range(1, 13)]
                })
                
                # Tính tổng sản lượng theo tháng
                monthly_sum = filtered_df.groupby('MONTH_NUM')['SL'].sum().reset_index()
                
                # Merge với DataFrame đầy đủ 12 tháng
                monthly_sum = pd.merge(all_months, monthly_sum, on='MONTH_NUM', how='left')
                
                # Vẽ biểu đồ cột
                fig = px.bar(
                    monthly_sum,
                    x='Month',
                    y='SL',
                    text='SL'
                )

                fig.update_layout(
                    title=None,
                    annotations=[dict(
                        text=f"<b>Tổng sản lượng theo tháng - {td} - Năm {year}</b>",
                        xref="paper", yref="paper",
                        x=0.45, y=1.1,
                        xanchor="center", yanchor="top",
                        showarrow=False,
                        font=dict(size=18, color="black")
                    )],
                    template="plotly_white",
                    height=600,
                    hovermode="x unified",
                    # xaxis_title="None",
                    yaxis_title="Sản lượng (triệu kWh)",
                    xaxis=dict(
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        title_font=dict(size=14, color="black", weight='bold'),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8,
                        categoryorder='array',
                        categoryarray=[f'T{i}' for i in range(1, 13)],  # Sắp xếp tháng theo thứ tự
                        title=None
                    ),
                    yaxis=dict(
                        showgrid=True,
                        gridcolor="lightgrey",
                        griddash="dot",
                        rangemode="tozero",
                        title_font=dict(size=14, color="black", weight="bold"),
                        ticklabelposition="outside",
                        ticks="outside",
                        ticklen=8
                    ),
                    margin=dict(l=60, r=40, t=80, b=80),
                    paper_bgcolor="white",
                    plot_bgcolor="#f0f8ff"
                )
                fig.add_annotation(
                    text="<b>Tháng</b>",
                    xref="paper",
                    yref="paper",
                    x=0.44,
                    y=-0.15,
                    showarrow=False,
                    font=dict(size=14, color="black"),
                )
                fig.update_traces(
                    marker_color="#3366ff",
                    texttemplate='%{text:.0f}',
                    textposition='outside'
                )

                fig.show()

            except Exception as e:
                print(f"❌ Lỗi khi vẽ biểu đồ: {e}")

    def on_change_day(change):
        if all(w.value is not None for w in day_widgets):
            plot_daily(td_dropdown_day.value, year_dropdown_day.value, month_dropdown_day.value)

    def on_change_month(change):
        if all(w.value is not None for w in month_widgets):
            plot_monthly(td_dropdown_month.value, year_dropdown_month.value)

    # Gắn observer cho các widget
    for w in day_widgets:
        w.observe(on_change_day, names='value')
    for w in month_widgets:
        w.observe(on_change_month, names='value')

    # Tạo layout cho widgets
    day_layout = widgets.HBox(
        day_widgets,
        layout=widgets.Layout(
            display='flex',
            flex_flow='row wrap',
            align_items='flex-start',
            width='100%',
            overflow='visible'
        )
    )

    month_layout = widgets.HBox(
        month_widgets,
        layout=widgets.Layout(
            display='flex',
            flex_flow='row wrap',
            align_items='flex-start',
            width='100%',
            overflow='visible'
        )
    )

    # Tạo tabs
    tab_daily = widgets.VBox([day_layout, out_day])
    tab_monthly = widgets.VBox([month_layout, out_month])
    
    tabs = widgets.Tab(children=[tab_daily, tab_monthly])
    tabs.set_title(0, 'SL theo ngày')
    tabs.set_title(1, 'SL theo tháng')
    
    # Thêm hàm xử lý sự kiện khi chuyển tab
    def on_tab_change(change):
        if change['new'] == 0:  # Tab SL theo ngày
            if all(w.value is not None for w in day_widgets):
                on_change_day(None)
        elif change['new'] == 1:  # Tab SL theo tháng
            if all(w.value is not None for w in month_widgets):
                on_change_month(None)
    
    # Gắn observer cho sự kiện chuyển tab
    tabs.observe(on_tab_change, names='selected_index')
    
    # Hiển thị giao diện
    print("✅ Giao diện sẵn sàng!")
    display(tabs)

    # Vẽ biểu đồ ban đầu cho tab đang active
    on_change_day(None)
    on_change_month(None)

Có 2 giá trị unique của cột 'Nhà máy' tương ứng với giá trị null trong cột 'TD_THAMCHIEU' của df_sanluong: ['Nậm Pảng 2' None]
🔄 Xử lý thời gian và tối ưu DataFrame...
✅ Đã xử lý thời gian và tối ưu index DataFrame.
🔄 Tạo giao diện tương tác...
✅ Giao diện sẵn sàng!
